# Setup

In [ ]:
# Install dependencies
from IPython.display import clear_output

!pip3 install gym
clear_output()

!rm -rf jelly-bean-world/ || true
!git clone https://github.com/NishanthVAnand/jelly-bean-world.git
clear_output()

%cd jelly-bean-world/
clear_output()

!rm -rf jbw/deps/core
!git rm jbw/deps/core
!git submodule add https://github.com/asaparov/core.git jbw/deps/core
clear_output()

!rm -rf jbw/deps/math
!git rm jbw/deps/math
!git submodule add https://github.com/asaparov/math.git jbw/deps/math
clear_output()

!git submodule update --init --recursive

%cd api/python/
!python setup.py install
clear_output()

%cd ../../..

import jbw
import gym

/content


In [ ]:
# Mount drive and create shortcut

from google.colab import drive
import sys
import os
import shutil
import warnings

drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/COMP579/Final_Project/" #@param {type:"string"}
!ln -Ts $folder /content/project 2> /dev/null

# Add the assignment folder to Python path
if '/content/project' not in sys.path:
    sys.path.insert(0, '/content/project')

Mounted at /content/gdrive


In [ ]:
%cd project/

/content/gdrive/MyDrive/COMP579/Final_Project


In [ ]:
import numpy as np
import sys
print(np.__version__)
print(sys.version)

1.21.5
3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]


# Training Code

In [ ]:
from dataclasses import dataclass
from os import listdir
from os.path import isfile, join
import importlib
import random
import numpy as np
import tensorflow as tf
import torch

from environments import JellyBeanEnv, MujocoEnv

def evaluate_agent(agent, env, n_episodes_to_evaluate):
  '''Evaluates the agent for a provided number of episodes.'''
  array_of_acc_rewards = []
  for i in range(n_episodes_to_evaluate):
    acc_reward = 0
    done = False
    curr_obs = env.reset()
    while not done:
      action = agent.act(curr_obs, mode='eval')
      next_obs, reward, done, _ = env.step(action)
      acc_reward += reward
      curr_obs = next_obs
    array_of_acc_rewards.append(acc_reward)
  return np.mean(np.array(array_of_acc_rewards))

def get_environment(env_type):
  '''Generates an environment specific to the agent type.'''
  if 'jellybean' in env_type:
    env = JellyBeanEnv(gym.make('JBW-COMP579-obj-v1'))
  elif 'mujoco' in env_type:
    env = MujocoEnv(gym.make('Hopper-v2'))
  else:
    raise Exception("ERROR: Please define your env_type to be either 'jellybean' or 'mujoco'!")
  return env

def train_agent(agent,
                env,
                env_eval,
                total_timesteps,
                evaluation_freq,
                n_episodes_to_evaluate):

  timestep = 0
  array_of_mean_acc_rewards = []
  best_reward = 0
    
  while timestep < total_timesteps:
    done = False
    curr_obs = env.reset()
    while not done:    
      action = agent.act(curr_obs, mode='train')
      next_obs, reward, done, _ = env.step(action)
      agent.update(curr_obs, action, reward, next_obs, done, timestep)
      curr_obs = next_obs
        
      timestep += 1
      if timestep % evaluation_freq == 0:
        mean_acc_rewards = evaluate_agent(agent, env_eval, n_episodes_to_evaluate)
        print('timestep: {ts}, acc_reward: {acr:.2f}'.format(ts=timestep, acr=mean_acc_rewards))
        array_of_mean_acc_rewards.append(mean_acc_rewards)
        if best_reward < mean_acc_rewards:
          best_reward = mean_acc_rewards
          print("Saving weights...")
          agent.save_weights()

  print('Done')

  return array_of_mean_acc_rewards

In [ ]:
@dataclass
class Arguments:
  group: str = 'GROUP_011'
  seed: int = 0
  pretrain: bool = False

args = Arguments()

seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
tf.random.set_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

path = './'+args.group+'/'
files = [f for f in listdir(path) if isfile(join(path, f))]
if ('agent.py' not in files) or ('env_info.txt' not in files):
  print("Your GROUP folder does not contain agent.py or env_info.txt!")
  exit()

with open(path+'env_info.txt') as f:
  lines = f.readlines()
env_type = lines[0].lower()

env = get_environment(env_type) 
env_eval = get_environment(env_type)

env.seed(seed)
env_eval.seed(seed)

if 'jellybean' in env_type:
  env_specs = {'scent_space': env.scent_space, 'vision_space': env.vision_space, 'feature_space': env.feature_space, 'action_space': env.action_space}
if 'mujoco' in env_type:
  env_specs = {'observation_space': env.observation_space, 'action_space': env.action_space}
agent_module = importlib.import_module(args.group+'.agent')
agent = agent_module.Agent(env_specs)
if args.pretrain:
  agent.load_weights()

# Note these can be environment specific and you are free to experiment with what works best for you
total_timesteps = 2e+6
evaluation_freq = 2000
n_episodes_to_evaluate = 1

learning_curve = train_agent(agent, env, env_eval, total_timesteps, evaluation_freq, n_episodes_to_evaluate)


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


timestep: 2000, acc_reward: 23.10
Saving weights...
timestep: 4000, acc_reward: 68.00
Saving weights...
timestep: 6000, acc_reward: 26.30
timestep: 8000, acc_reward: 42.90
timestep: 10000, acc_reward: 19.30
timestep: 12000, acc_reward: 171.40
Saving weights...
timestep: 14000, acc_reward: 41.70
timestep: 16000, acc_reward: 27.00
timestep: 18000, acc_reward: 9.00
timestep: 20000, acc_reward: 4.00
timestep: 22000, acc_reward: 6.00
timestep: 24000, acc_reward: 119.60
timestep: 26000, acc_reward: 26.90
timestep: 28000, acc_reward: 89.60
timestep: 30000, acc_reward: 6.00
timestep: 32000, acc_reward: 17.00
timestep: 34000, acc_reward: 29.90
timestep: 36000, acc_reward: 23.90
timestep: 38000, acc_reward: 23.90
timestep: 40000, acc_reward: 23.90
timestep: 42000, acc_reward: 36.00
timestep: 44000, acc_reward: 64.10
timestep: 46000, acc_reward: 78.40
timestep: 48000, acc_reward: 98.60
timestep: 50000, acc_reward: 111.00
timestep: 52000, acc_reward: 56.00
timestep: 54000, acc_reward: 33.90
timest